## Linear Regression Neural Network Model Implementation Colorectal Adenocarcinoma

#### Initializing Libraries and Importing Packages

In [113]:
import pandas as pd
import numpy as py
import statistics as stat
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error # Performance metrics

#### Importing Colorectal Patient Dataset

In [114]:
# set our dataset equal to ds variable
ds = pd.read_csv('Colorectal Dataset Modified Garik Kazanjian.csv')

# view first 5 lines of dataset
ds.head()

,Patient ID,Diagnosis Age,Neoplasm Disease Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Publication Version Type,Aneuploidy Score,Buffa Hypoxia Score,TCGA PanCanAtlas Cancer Type Acronym,Cancer Type Detailed,Last Communication Contact from Initial Pathologic Diagnosis Date,Birth from Initial Pathologic Diagnosis Date,...,Ragnum Hypoxia Score,Sex,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site Code,TMB (nonsynonymous),Tumor Disease Anatomic Site,Tumor Type,Patient Weight,Medicinal Treatment
0,TCGA-A6-2671,85,STAGE IV,6TH,25.0,NaN,COAD,Colon Adenocarcinoma,1126.0,-31329.0,...,NaN,Male,Yes,No,A6,4.066667,Colon,Colon Adenocarcinoma,67.2,Fluorouacil
1,TCGA-A6-2674,71,STAGE IV,6TH,24.0,NaN,COAD,Mucinous Adenocarcinoma of the Colon and Rectum,1331.0,-26292.0,...,NaN,Male,Yes,No,A6,7.666667,Colon,"Colon Adenocarcinoma, Mucinous Type",85.6,Fluorouacil
2,TCGA-A6-2676,75,STAGE IIB,6TH,1.0,NaN,COAD,Colon Adenocarcinoma,711.0,-27403.0,...,NaN,Female,Yes,No,A6,NaN,Colon,Colon Adenocarcinoma,45.9,Fluorouacil
3,TCGA-A6-2677,68,STAGE IIIC,6TH,8.0,NaN,COAD,Colon Adenocarcinoma,541.0,-25143.0,...,NaN,Female,Yes,No,A6,7.766667,Colon,Colon Adenocarcinoma,55.2,Fluorouacil
4,TCGA-A6-2678,43,STAGE IIIB,6TH,7.0,NaN,COAD,Colon Adenocarcinoma,1286.0,-16030.0,...,NaN,Female,Yes,No,A6,1.733333,Colon,Colon Adenocarcinoma,96.1,Fluorouacil


#### Dataset Cleaning

In [115]:
# dropping all columns which are not included in our feature set
columns_drop = ['Patient ID', 'Oncotree Code', 'Progression Free Status', 'Tumor Type', 'Sex', 'Informed consent verified', 'Primary Lymph Node Presentation Assessment', 'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code', 'American Joint Committee on Cancer Publication Version Type', 'Cancer Type Detailed', 'International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code']
ds = ds.drop(columns=columns_drop)

# assigning each 'Cancer Stage Code'; COAD (colon/colon and rectum) 0; READ (rectum) 1
stage_dict = {
    '0:DiseaseFree': '0',
    '1:Recurred/Progressed': '1'}

# map each respective treatment with its number
ds['Disease Free Status'] = ds['Disease Free Status'].map(stage_dict)

# assigning each 'Cancer Stage Code'; COAD (colon/colon and rectum) 0; READ (rectum) 1
survival_dict = {
    '1:DEAD WITH TUMOR': '1',
    '0:ALIVE OR DEAD TUMOR FREE': '0'}

# map each respective treatment with its number
ds['Disease-specific Survival status'] = ds['Disease-specific Survival status'].map(survival_dict)


# assigning each 'Cancer Stage Code'; COAD (colon/colon and rectum) 0; READ (rectum) 1
gen_dict = {
    'EUR': '1',
    'EUR_ADMIX': '2',
    'AFR': '3',
    'AFR_ADMIX': '4'}

# map each respective treatment with its number
ds['Genetic Ancestry Label'] = ds['Genetic Ancestry Label'].map(gen_dict)




# assigning each 'Cancer Stage Code'; COAD (colon/colon and rectum) 0; READ (rectum) 1
acronym_dict = {
    'COAD': '0',
    'READ': '1'}

# map each respective treatment with its number
ds['TCGA PanCanAtlas Cancer Type Acronym'] = ds['TCGA PanCanAtlas Cancer Type Acronym'].map(acronym_dict)

# assigning each 'Cancer Stage Code' with an appropriate number according to its stage
stage_dict = {
    'STAGE I': '1',
    'STAGE II': '2',
    'STAGE IIA': '2.25',
    'STAGE IIB': '2.50',
    'STAGE IIC': '2.75',
    'STAGE III': '3',
    'STAGE IIIA': '3.25',
    'STAGE IIIB': '3.50',
    'STAGE IIIC': '3.75',
    'STAGE IV': '4',
    'STAGE IVA': '4.25',
    'STAGE IVB': '4.75',
    'NA': '0'}

# map each respective treatment with its number
ds['Neoplasm Disease Stage American Joint Committee on Cancer Code'] = ds['Medicinal Treatment'].map(stage_dict)

# assigning each treatment with an appropriate number according to its position within the column
treatment_dict = {
    'Fluorouacil': '1',
    'Leucovorin': '2',
    'Oxaliplatin': '3',
    'Bevacizumab': '4',
    'Radiation 1': '5'}

# map each respective treatment with its number
ds['Medicinal Treatment'] = ds['Medicinal Treatment'].map(treatment_dict)



# assigning each treatment with an appropriate number according to its position within the column
icd_dict = {
    'C80.1': '80.1',
    'C49.4': '49.4',
    'C20': '20',
    'C19': '19',
    'C18.9': '18.9',
    'C18.7': '18.7',
    'C18.6': '18.6',
    'C18.5': '18.5',
    'C18.4': '18.4',
    'C18.3': '18.3',
    'C18.2': '18.2',
    'C18.0': '18.0'}

# map each respective treatment with its number
ds['ICD-10 Classification'] = ds['ICD-10 Classification'].map(icd_dict)


# International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code

# assigning each treatment with an appropriate number according to its position within the column
hist_dict = {
    'C80.1': '80.1',
    'C49.4': '49.4',
    'C20': '20',
    'C19': '19',
    'C18.9': '18.9',
    'C18.7': '18.7',
    'C18.6': '18.6',
    'C18.5': '18.5',
    'C18.4': '18.4',
    'C18.3': '18.3',
    'C18.2': '18.2',
    'C18.0': '18.0'}

# map each respective treatment with its number
ds['ICD-10 Classification'] = ds['ICD-10 Classification'].map(hist_dict)




# assigning each treatment with an appropriate number according to its position within the column
neo_dict = {
    'Yes': '1',
    'No': '0'}

# map each respective treatment with its number
ds['New Neoplasm Event Post Initial Therapy Indicator'] = ds['New Neoplasm Event Post Initial Therapy Indicator'].map(neo_dict)

#Tissue Prospective Collection Indicator
# assigning each treatment with an appropriate number according to its position within the column
tiss_dict = {
    'Yes': '1',
    'No': '0'}

# map each respective treatment with its number
ds['Tissue Prospective Collection Indicator'] = ds['Tissue Prospective Collection Indicator'].map(tiss_dict)

# assigning each treatment with an appropriate number according to its position within the column
retro_dict = {
    'Yes': '1',
    'No': '0'}

# map each respective treatment with its number
ds['Tissue Retrospective Collection Indicator'] = ds['Tissue Retrospective Collection Indicator'].map(retro_dict)

# assigning each treatment with an appropriate number according to its position within the column
live_dict = {
    '1:DECEASED': '1',
    '0:LIVING': '0'}

# map each respective treatment with its number
ds['Overall Survival Status'] = ds['Overall Survival Status'].map(live_dict)




# assigning each treatment with an appropriate number according to its position within the column
mest_dict = {
    'M0': '0',
    'M1': '1',
    'M1A': '1.25',
    'M1B': '1.75',
    'MX': '5'}

# map each respective treatment with its number
ds['American Joint Committee on Cancer Metastasis Stage Code'] = ds['American Joint Committee on Cancer Metastasis Stage Code'].map(mest_dict)


# Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code
# assigning each treatment with an appropriate number according to its position within the column
lymh_dict = {
    'NX': '8',
    'N2B': '7',
    'N2A': '6',
    'N2': '5',
    'N1C': '4',
    'N1B': '3',
    'N1A': '2',
    'N1': '1',}

# map each respective treatment with its number
ds['Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code'] = ds['Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code'].map(lymh_dict)

# assigning each treatment with an appropriate number according to its position within the column
tum_dict = {
    'With Tumor': '0',
    'Tumor Free': '1',
    'NA': '2'}

# map each respective treatment with its number
ds['Person Neoplasm Cancer Status'] = ds['Person Neoplasm Cancer Status'].map(tum_dict)


# Progression Free Status

# assigning each treatment with an appropriate number according to its position within the column
#prog_dict = {
 #   '1:PROGRESSION': '1',
  #  '0:CENSORED': '0'}

# map each respective treatment with its number
#ds['Progression Free Status'] = ds['Progression Free Status'].map(prog_dict)

 # assigning each treatment with an appropriate number according to its position within the column
#sex_dict = {
 #   'Male': '1',
  #  'Female': '0'}

# map each respective treatment with its number
#ds['Sex'] = ds['Sex'].map(sex_dict)

# assigning each treatment with an appropriate number according to its position within the column
code_dict = {
    'G5': '8',
    'F5': '7',
    'El': '6',
    'DY': '5',
    'DT': '4',
    'DC': '3',
    'D5': '2',
    'CM': '1',
    'CL': '7',
    'CA': '6',
    'AZ': '5',
    'AY': '4',
    'AU': '3',
    'AM': '2',
    'AH': '1',
    'AG': '7',
    'AF': '6',
    'AD': '5',
    'AA': '4',
    'A6': '3',
    '5M': '2',
    '4T': '6',
    '4N': '5',
    '3L': '4'}

# map each respective treatment with its number
ds['Tissue Source Site Code'] = ds['Tissue Source Site Code'].map(code_dict)



# Tumor Disease Anatomic Site
ana_dict = {
    'Colon': '1',
    'Rectum': '0'}

# map each respective treatment with its number
ds['Tumor Disease Anatomic Site'] = ds['Tumor Disease Anatomic Site'].map(ana_dict)


# this for loop will iterate throughout the columns, check for empty values, and replace then with a -1
for column in ds:
    ds[column] = ds[column].replace(py.NaN,1)
    
# Set display option to show all columns
#pd.set_option('display.max_columns', None)
#
# Display the DataFrame
#print(ds)
# Attempt to convert all columns to numeric, coercing errors
for col in ds.columns:
    ds[col] = pd.to_numeric(ds[col], errors='ignore')




ds = ds[:-175]
ds.head()


,Diagnosis Age,Neoplasm Disease Stage American Joint Committee on Cancer Code,Aneuploidy Score,Buffa Hypoxia Score,TCGA PanCanAtlas Cancer Type Acronym,Last Communication Contact from Initial Pathologic Diagnosis Date,Birth from Initial Pathologic Diagnosis Date,Disease Free (Months),Disease Free Status,Months of disease-specific survival,...,Person Neoplasm Cancer Status,Progress Free Survival (Months),Ragnum Hypoxia Score,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site Code,TMB (nonsynonymous),Tumor Disease Anatomic Site,Patient Weight,Medicinal Treatment
0,85,1,25.0,1.0,0,1126.0,-31329.0,1.000000,1,43.758425,...,0,17.588848,1.0,1,0,3,4.066667,1,67.2,1
1,71,1,24.0,1.0,0,1331.0,-26292.0,1.000000,1,43.758425,...,1,24.164119,1.0,1,0,3,7.666667,1,85.6,1
2,75,1,1.0,1.0,0,711.0,-27403.0,42.903639,0,42.903639,...,1,42.903639,1.0,1,0,3,1.000000,1,45.9,1
3,68,1,8.0,1.0,0,541.0,-25143.0,1.000000,1,24.328501,...,1,24.328501,1.0,1,0,3,7.766667,1,55.2,1
4,43,1,7.0,1.0,0,1286.0,-16030.0,42.278989,0,42.278989,...,1,42.278989,1.0,1,0,3,1.733333,1,96.1,1


#### Initializing Training and Testing Set

In [116]:
# Assuming `ds` is your DataFrame
train_x = py.asarray(ds.drop(columns='Medicinal Treatment'))
train_y = py.asarray(ds['Medicinal Treatment'])

train_dataset = ds.sample(frac=0.8, random_state=0)
test_dataset = ds.drop(train_dataset.index)

#### Initializing Weights

In [117]:
# Initialize weights (a should match the number of features in train_x)
a = tf.Variable(initial_value=0.0, dtype=tf.float64)
b = tf.Variable(initial_value=0.0, dtype=tf.float64)

#### Building Model and Loss Function

In [118]:
def h(x):
    # Compute dot product and add bias
    y = tf.reduce_sum(a * x, axis=1) + b
    return y

def loss_object(y, train_y):
    return tf.reduce_mean(tf.square(y - train_y))

learning_rate = 0.01
train_data = []
loss_values = []
a_values = []
b_values = []
mse = []
r2 = []
mae = []
training_epochs = 200

#### Model Training

In [119]:
# Train model
for epoch in range(training_epochs):
    with tf.GradientTape() as tape:

        train_x = tf.cast(train_x, tf.float64)
        y_predicted = h(train_x)  # Ensure `train_x` is cast before passing to `h`

        # Check the data type of y_predicted (output of h(train_x))

        y_predicted = h(train_x)
        loss_value = loss_object(train_y, y_predicted)
        loss_values.append(loss_value.numpy())

        # Get gradients
        gradients = tape.gradient(loss_value, [a, b])

        gradients = [tf.cast(g, tf.float64) for g in gradients]

        # Save current values of a and b
        a_values.append(a.numpy())
        b_values.append(b.numpy())

        # Update weights and bias
        a.assign_sub(gradients[0] * learning_rate)
        b.assign_sub(gradients[1] * learning_rate)

        if epoch % 5 == 0:
            train_data.append([a.numpy(), b.numpy()])


        print(f'prediction ---------------------------------------------------- \n{y_predicted}')
        print(f'Mean Squared Error: {mean_squared_error(train_y, y_predicted)}')
# .abs(predictions - test_y))
        
        mse.append(mean_squared_error(train_y, y_predicted))
        r2.append(r2_score(train_y, y_predicted))
        mae.append(median_absolute_error(train_y, y_predicted))


        print(f'Average of Mean Squared Error Values: {stat.mean(mse)}')
        print(f'Average of R^2 Values: {stat.mean(mse)}')
        print(f'Average of Median Absolute Error Values: {stat.mean(mae)}')

print("Training complete.")

prediction ---------------------------------------------------- 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:5

prediction ---------------------------------------------------- 
[-1.80697369e+167 -1.48491312e+167 -1.59710438e+167 -1.46916848e+167
 -8.71550689e+166 -1.53804050e+167 -1.22162888e+167 -1.10093291e+167
 -1.18586052e+167 -1.36267872e+167 -1.54168554e+167 -9.62261200e+166
 -1.72075249e+167 -8.17868111e+166 -1.14192330e+167 -1.31986814e+167
 -6.62441359e+166 -1.17314691e+167 -1.18180046e+167 -1.39744687e+167
 -1.15655667e+167 -1.24311233e+167 -1.37662358e+167 -7.85530490e+166
 -1.19811780e+167 -1.13991394e+167 -1.27504270e+167 -1.20915338e+167
 -1.25222954e+167 -1.50021207e+167 -1.50733305e+167 -1.54766630e+167
 -1.23872967e+167 -1.26908423e+167 -1.47940230e+167 -1.02577536e+167
 -1.39996458e+167 -1.38052760e+167 -1.05646562e+167 -1.68360695e+167
 -1.68770778e+167 -1.51321125e+167 -1.25958898e+167 -1.50946834e+167
 -1.48475617e+167 -1.40103632e+167 -1.73594739e+167 -1.37198573e+167
 -1.06639039e+167 -4.79273789e+166 -1.52146265e+167 -1.56616750e+167
 -8.70272821e+166 -1.32422118e+167 -1.

C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)


prediction ---------------------------------------------------- 
[1.61344816e+174 1.32588004e+174 1.42605570e+174 1.31182163e+174
 7.78208261e+173 1.37331751e+174 1.09079334e+174 9.83023821e+173
 1.05885574e+174 1.21673685e+174 1.37657217e+174 8.59203744e+173
 1.53646119e+174 7.30275047e+173 1.01962416e+174 1.17851124e+174
 5.91494385e+173 1.04750375e+174 1.05523051e+174 1.24778135e+174
 1.03269031e+174 1.10997593e+174 1.22918822e+174 7.01400762e+173
 1.06980028e+174 1.01783001e+174 1.13848658e+174 1.07965396e+174
 1.11811669e+174 1.33954047e+174 1.34589880e+174 1.38191239e+174
 1.10606265e+174 1.13316626e+174 1.32095942e+174 9.15915582e+173
 1.25002942e+174 1.23267412e+174 9.43318939e+173 1.50329390e+174
 1.50695553e+174 1.35114746e+174 1.12468794e+174 1.34780541e+174
 1.32573989e+174 1.25098637e+174 1.55002872e+174 1.22504708e+174
 9.52180779e+173 4.27943924e+173 1.35851514e+174 1.39843214e+174
 7.77067252e+173 1.18239808e+174 1.27444100e+174 1.16763208e+174
 1.03700800e+174 7.723143

...

prediction ---------------------------------------------------- 
[-2.35184764e+292 -1.93267309e+292 -2.07869444e+292 -1.91218082e+292
 -1.13435765e+292 -2.00182047e+292 -1.58999825e+292 -1.43290768e+292
 -1.54344431e+292 -1.77358019e+292 -2.00656463e+292 -1.25242096e+292
 -2.23962735e+292 -1.06448765e+292 -1.48625828e+292 -1.71786052e+292
 -8.62193598e+291 -1.52689705e+292 -1.53815998e+292 -1.81883231e+292
 -1.50530419e+292 -1.61795980e+292 -1.79172997e+292 -1.02239896e+292
 -1.55939765e+292 -1.48364302e+292 -1.65951845e+292 -1.57376089e+292
 -1.62982621e+292 -1.95258527e+292 -1.96185350e+292 -2.01434882e+292
 -1.61225560e+292 -1.65176326e+292 -1.92550053e+292 -1.33508715e+292
 -1.82210921e+292 -1.79681120e+292 -1.37503174e+292 -2.19128095e+292
 -2.19661834e+292 -1.96950422e+292 -1.63940481e+292 -1.96463267e+292
 -1.93246880e+292 -1.82350411e+292 -2.25940410e+292 -1.78569363e+292
 -1.38794922e+292 -6.23793769e+291 -1.98024374e+292 -2.03842888e+292
 -1.13269445e+292 -1.72352617e+292 -1.

C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)


prediction ---------------------------------------------------- 
[2.09996652e+299 1.72568525e+299 1.85606782e+299 1.70738768e+299
 1.01286879e+299 1.78742699e+299 1.41971062e+299 1.27944434e+299
 1.37814258e+299 1.58363108e+299 1.79166306e+299 1.11828761e+299
 1.99976493e+299 9.50481822e+298 1.32708113e+299 1.53387894e+299
 7.69853308e+298 1.36336751e+299 1.37342420e+299 1.62403672e+299
 1.34408724e+299 1.44467752e+299 1.59983703e+299 9.12900799e+298
 1.39238732e+299 1.32474596e+299 1.48178526e+299 1.40521228e+299
 1.45527304e+299 1.74346484e+299 1.75174046e+299 1.79861357e+299
 1.43958423e+299 1.47486065e+299 1.71928086e+299 1.19210031e+299
 1.62696267e+299 1.60437406e+299 1.22776686e+299 1.95659640e+299
 1.96136216e+299 1.75857179e+299 1.46382578e+299 1.75422198e+299
 1.72550284e+299 1.62820818e+299 2.01742361e+299 1.59444717e+299
 1.23930090e+299 5.56985923e+298 1.76816112e+299 1.82011467e+299
 1.01138373e+299 1.53893781e+299 1.65873531e+299 1.51971928e+299
 1.34970687e+299 1.005197

C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\armkz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p

ValueError: Input contains NaN.

#### Model Predictions

In [ ]:
#predictions = h(test_x)
#print(predictions)

#### Model Evaluation

In [ ]:
#mse = tf.reduce_mean(tf.square(predictions - test_y))
#print(f"Mean Squared Error: {mse.numpy()}")

#mae = tf.reduce_mean(tf.abs(predictions - test_y))
#print(f"Mean Absolute Error: {mae.numpy()}")

#r2 = r2_score(test_y, predictions)
#print(f"R² Score: {r2.numpy()}")

#import matplotlib.pyplot as plt

#hist = pd.DataFrame(history.history)
#hist['epoch'] = history.epoch
#hist.tail()